# Lab 6 Report

Prepared by Zech Wolf

## 1. Data preparation

The dataset used is the handwritten letters dataset from EMNIST. There are 26 classes for each of the letters in the Roman alphabet. The full dataset contains over 145,000 instances. The provided training set is used as the full dataset for this report (about 88,000 instances). Each instance is a 28x28 grayscale image containing a handwritten letter.

Dataset source: https://www.kaggle.com/datasets/crawford/emnist?resource=download&select=emnist-letters-train.csv (processed into .csv format from original source)

### 1.1 Performance metrics

A possible business case for this dataset is optical character recognition (OCR), which converts text from an image into digital text on a computer. A classification model trained on this dataset could be used to process handwriting.

The proper metric is accuracy, since all classes are equally important, and the dataset is balanced.

### 1.2 Cross validation method

With nearly 90,000 instances, the dataset is very large. The dataset is also balanced by class. For these reasons, a stratified 80/20 split should be suitable for training and testing.

In [47]:
import pandas as pd

df = pd.read_csv("datasets/emnist-letters-train.csv")

In [77]:
import numpy as np
import tensorflow as tf

X = df.drop(columns=df.columns[0]).to_numpy(dtype=np.float32)
X = X.reshape((len(X),28,28,1))  #reshape to tensor format

y = df[df.columns[0]].to_numpy()

In [79]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=1234)
X_train = tf.image.per_image_standardization(X_train)
X_test = tf.image.per_image_standardization(X_test) #normalization step

## 2. Modeling

### 2.1 Data augmentation

Random rotations and random brightness changes are applied to the dataset as preprocessing steps